# Predictive Algorithms

In [ ]:
%load_ext autoreload
%autoreload 2

## Load and Preprocess Data

This step will involve loading the data, handling missing values, and encoding categorical variables to prepare them for modeling.

In [ ]:
from data_processing import *
from lookups import *

db_config_path = '/home/jupyter/.my.cnf'
db_name = 'actin_personalization'
query = "SELECT * FROM knownPalliativeTreatments"

preprocessor = DataPreprocessor(db_config_path, db_name)

In [ ]:
pfs_df, pfs_features, pfs_encoded_columns = preprocessor.preprocess_data(query, duration_col = "observedPfsDays", event_col = "hadProgressionEvent", features = features)

In [ ]:
os_df, os_features, os_encoded_columns = preprocessor.preprocess_data(query, duration_col = "observedOsFromTreatmentStartDays", event_col = "hadSurvivalEvent", features = features)

## Models

In [ ]:
from survival_models import *
from model_trainer import *

from sksurv.util import Surv
from lookups import features

splitter = DataSplitter(test_size=0.1, random_state=42)

# y_pfs = Surv.from_dataframe(event='hadProgressionEvent', time='observedPfsDays', data=pfs_df)

y_os = Surv.from_dataframe(event='hadSurvivalEvent', time='observedOsFromTreatmentStartDays', data=os_df)

X_train, X_test, y_train, y_test = splitter.split(os_df[os_features], y_os, 'systemicTreatmentPlan', os_encoded_columns)

models = {
    'DeepSurv': DeepSurv(input_size=X_train.shape[1]),
    'AalenAdditive': AalenAdditiveModel(),
    'CoxPH': CoxPHModel(),
    'RandomSurvivalForest': RandomSurvivalForestModel(),
    'GradientBoosting': GradientBoostingSurvivalModel(),
}

trainer = ModelTrainer(models=models, n_splits=5, random_state=42)

results = trainer.train_and_evaluate(
    X_train,
    y_train,
    X_test,
    y_test,
    treatment_col='systemicTreatmentPlan',
    encoded_columns=os_encoded_columns,
    event_col='hadSurvivalEvent',
    duration_col='observedOsFromTreatmentStartDays'
)

print("\nFinal Results:") 
for model_name, metrics in results.items():
    print(f"{model_name}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")